In [30]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Connecting the sql database to python

In [31]:
football = mysql.connector.connect(
    host='localhost', 
    user='root',
    password='rootpassword',
    database='football_db'
)

In [32]:
mycursor = football.cursor()

# Storing the tables in a dataframe for each

In [33]:
df_games = pd.read_sql_query("SELECT * FROM games", football)
df_game_lineups = pd.read_sql_query("SELECT * FROM game_lineups", football)
df_game_events = pd.read_sql_query("SELECT * FROM game_events", football)
df_appearances = pd.read_sql_query("SELECT * FROM appearances", football)
df_players = pd.read_sql_query("SELECT * FROM players", football)

# Merging The dataframes

In [34]:
df_A = pd.merge(df_games, df_game_lineups, on='game_id', how='outer')
df_B = pd.merge(df_A, df_game_events, on='game_id', how='outer')
df_C = pd.merge(df_B, df_appearances, on='game_id', how='outer')
df = pd.merge(df_C, df_players, on='player_id', how='outer')

df.head()

,MyUnknownColumn_x,game_id,competition_id_x,season,round,date_x,home_club_goals,away_club_goals,home_club_position,away_club_position,...,country_of_birth,date_of_birth,sub_position,position_y,foot,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name
0,0.0,2222734.0,RU1,2012.0,19. Matchday,07-12-2012,2.0,3.0,11.0,10.0,...,UdSSR,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport
1,117.0,2598832.0,UKR1,2015.0,9. Matchday,27-09-2015,0.0,0.0,11.0,5.0,...,UdSSR,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport
2,154.0,2222798.0,RU1,2012.0,24. Matchday,14-04-2013,1.0,1.0,4.0,11.0,...,UdSSR,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport
3,293.0,2619433.0,UKRP,2015.0,last 16 1st leg,23-09-2015,0.0,1.0,9.0,9.0,...,UdSSR,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport
4,485.0,2598841.0,UKR1,2015.0,10. Matchday,03-10-2015,2.0,0.0,2.0,11.0,...,UdSSR,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport


# Combining the competition_x and competition_y to a new column as they are the same and then dropping them after combining.

In [35]:
df['competition_id'] = df['competition_id_x'].combine_first(df['competition_id_y'])
df = df.drop(columns = ['competition_id_x', 'competition_id_y'])
df.head()

,MyUnknownColumn_x,game_id,season,round,date_x,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,...,date_of_birth,sub_position,position_y,foot,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,competition_id
0,0.0,2222734.0,2012.0,19. Matchday,07-12-2012,2.0,3.0,11.0,10.0,Miodrag Bozovic,...,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport,RU1
1,117.0,2598832.0,2015.0,9. Matchday,27-09-2015,0.0,0.0,11.0,5.0,Oleksandr Babych,...,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport,UKR1
2,154.0,2222798.0,2012.0,24. Matchday,14-04-2013,1.0,1.0,4.0,11.0,Kurban Berdyev,...,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport,RU1
3,293.0,2619433.0,2015.0,last 16 1st leg,23-09-2015,0.0,1.0,9.0,9.0,Oleksandr Babych,...,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport,UKRP
4,485.0,2598841.0,2015.0,10. Matchday,03-10-2015,2.0,0.0,2.0,11.0,Mircea Lucescu,...,17-11-1988,Centre-Forward,Attack,right,175.0,1000000.0,500000.0,,Prosport,UKR1


# Combining the date_x, date_y and date column to a new column as they are the same and then dropping them after combining.

In [36]:
df['date_id'] = df['date_x'].combine_first(df['date_y']).combine_first(df['date'])
df = df.drop(columns = ['date_x', 'date_y', 'date'])

# Dropping the MyunknownColumn_x and MyunknownColumn_y as they are just indexes created during merging.

In [37]:
df = df.drop(columns = ['MyUnknownColumn_x', 'MyUnknownColumn_y'])


# Combining the player_id, player_x and player_id_y to a new column as they are the same and then dropping them after combining.

In [38]:
df['players_id'] = df['player_id'].combine_first(df['player_id_x']).combine_first(df['player_id_y'])
df = df.drop(columns = ['player_id', 'player_id_x', 'player_id_y'])

# Combining the player_name_x, player_name_y and name to a new column as they are the same and then dropping them after combining.

In [39]:
df['Player_name'] = df['player_name_x'].combine_first(df['player_name_y']).combine_first(df['name'])
df = df.drop(columns = ['player_name_x', 'player_name_y', 'name'])


# Combining the position_x and position_y to a new column as they are the same and then dropping them after combining.

In [40]:
df['players_position'] = df['position_y'].combine_first(df['position_x'])
df = df.drop(columns = ['position_y', 'position_x'])


# Changing the columns name

In [41]:
df = df.rename(columns = {'type_x': 'lineup_type', 'type_y': 'game_event_type', 'number': 'Players_shirt_number', 'stadium': 'Stadium_name', 'country_of_birth':'players_birth_country', 'last_season': 'players_last_season', 'season_of_the_game': 'season_of_the_game_years'})
df.head()

,game_id,season,round,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,Stadium_name,...,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,competition_id,date_id,players_id,Player_name,players_position
0,2222734.0,2012.0,19. Matchday,2.0,3.0,11.0,10.0,Miodrag Bozovic,Slavoljub Muslin,Olimp-2,...,175.0,1000000.0,500000.0,,Prosport,RU1,07-12-2012,104203.0,Eugene Starikov,Attack
1,2598832.0,2015.0,9. Matchday,0.0,0.0,11.0,5.0,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175.0,1000000.0,500000.0,,Prosport,UKR1,27-09-2015,104203.0,Eugene Starikov,Attack
2,2222798.0,2012.0,24. Matchday,1.0,1.0,4.0,11.0,Kurban Berdyev,Miodrag Bozovic,Central Stadium Kazan,...,175.0,1000000.0,500000.0,,Prosport,RU1,14-04-2013,104203.0,Eugene Starikov,Attack
3,2619433.0,2015.0,last 16 1st leg,0.0,1.0,9.0,9.0,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175.0,1000000.0,500000.0,,Prosport,UKRP,23-09-2015,104203.0,Eugene Starikov,Attack
4,2598841.0,2015.0,10. Matchday,2.0,0.0,2.0,11.0,Mircea Lucescu,Oleksandr Babych,Arena Lviv,...,175.0,1000000.0,500000.0,,Prosport,UKR1,03-10-2015,104203.0,Eugene Starikov,Attack


In [42]:
df = df.rename(columns = {'ï»¿game_event_id': 'game_event_id', 'ï»¿appearance_id': 'appearance_id'})

# Checking Data Types for each column

In [43]:
print(df.dtypes.to_string())

game_id                        float64
season                         float64
round                           object
home_club_goals                float64
away_club_goals                float64
home_club_position             float64
away_club_position             float64
home_club_manager_name          object
away_club_manager_name          object
Stadium_name                    object
attendance                     float64
referee                         object
home_club_formation             object
away_club_formation             object
home_club_name                  object
away_club_name                  object
aggregate                       object
competition_type                object
ï»¿game_lineups_id              object
lineup_type                     object
Players_shirt_number           float64
team_captain                   float64
game_event_id                   object
minute                         float64
game_event_type                 object
description              

In [44]:
df.shape

(4742, 51)

# Correcting the data types

In [45]:
columns = 'game_id','home_club_goals', 'away_club_goals', 'home_club_position', 'away_club_position', 'attendance', 'Players_shirt_number', 'current_club_id'


for col in columns:
    df[col] = df[col].fillna(0).astype(int)
df.head()

,game_id,season,round,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,Stadium_name,...,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,competition_id,date_id,players_id,Player_name,players_position
0,2222734,2012.0,19. Matchday,2,3,11,10,Miodrag Bozovic,Slavoljub Muslin,Olimp-2,...,175.0,1000000.0,500000.0,,Prosport,RU1,07-12-2012,104203.0,Eugene Starikov,Attack
1,2598832,2015.0,9. Matchday,0,0,11,5,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175.0,1000000.0,500000.0,,Prosport,UKR1,27-09-2015,104203.0,Eugene Starikov,Attack
2,2222798,2012.0,24. Matchday,1,1,4,11,Kurban Berdyev,Miodrag Bozovic,Central Stadium Kazan,...,175.0,1000000.0,500000.0,,Prosport,RU1,14-04-2013,104203.0,Eugene Starikov,Attack
3,2619433,2015.0,last 16 1st leg,0,1,9,9,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175.0,1000000.0,500000.0,,Prosport,UKRP,23-09-2015,104203.0,Eugene Starikov,Attack
4,2598841,2015.0,10. Matchday,2,0,2,11,Mircea Lucescu,Oleksandr Babych,Arena Lviv,...,175.0,1000000.0,500000.0,,Prosport,UKR1,03-10-2015,104203.0,Eugene Starikov,Attack


In [46]:
columns2 = 'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played', 'team_captain', 'minute', 'height_in_cm', 'market_value_in_eur', 'highest_market_value_in_eur', 'players_id'

for col in columns2:
    df[col]=df[col].fillna(0).astype(int)
df.head()

,game_id,season,round,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,Stadium_name,...,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,competition_id,date_id,players_id,Player_name,players_position
0,2222734,2012.0,19. Matchday,2,3,11,10,Miodrag Bozovic,Slavoljub Muslin,Olimp-2,...,175,1000000,500000,,Prosport,RU1,07-12-2012,104203,Eugene Starikov,Attack
1,2598832,2015.0,9. Matchday,0,0,11,5,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175,1000000,500000,,Prosport,UKR1,27-09-2015,104203,Eugene Starikov,Attack
2,2222798,2012.0,24. Matchday,1,1,4,11,Kurban Berdyev,Miodrag Bozovic,Central Stadium Kazan,...,175,1000000,500000,,Prosport,RU1,14-04-2013,104203,Eugene Starikov,Attack
3,2619433,2015.0,last 16 1st leg,0,1,9,9,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175,1000000,500000,,Prosport,UKRP,23-09-2015,104203,Eugene Starikov,Attack
4,2598841,2015.0,10. Matchday,2,0,2,11,Mircea Lucescu,Oleksandr Babych,Arena Lviv,...,175,1000000,500000,,Prosport,UKR1,03-10-2015,104203,Eugene Starikov,Attack


In [47]:
columns3 = 'season', 'players_last_season'
for col in columns3:
    df[col]=df[col].fillna(0).astype(int)
df.head()

,game_id,season,round,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,Stadium_name,...,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,competition_id,date_id,players_id,Player_name,players_position
0,2222734,2012,19. Matchday,2,3,11,10,Miodrag Bozovic,Slavoljub Muslin,Olimp-2,...,175,1000000,500000,,Prosport,RU1,07-12-2012,104203,Eugene Starikov,Attack
1,2598832,2015,9. Matchday,0,0,11,5,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175,1000000,500000,,Prosport,UKR1,27-09-2015,104203,Eugene Starikov,Attack
2,2222798,2012,24. Matchday,1,1,4,11,Kurban Berdyev,Miodrag Bozovic,Central Stadium Kazan,...,175,1000000,500000,,Prosport,RU1,14-04-2013,104203,Eugene Starikov,Attack
3,2619433,2015,last 16 1st leg,0,1,9,9,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175,1000000,500000,,Prosport,UKRP,23-09-2015,104203,Eugene Starikov,Attack
4,2598841,2015,10. Matchday,2,0,2,11,Mircea Lucescu,Oleksandr Babych,Arena Lviv,...,175,1000000,500000,,Prosport,UKR1,03-10-2015,104203,Eugene Starikov,Attack


In [48]:
print(df.dtypes.to_string())

game_id                         int32
season                          int32
round                          object
home_club_goals                 int32
away_club_goals                 int32
home_club_position              int32
away_club_position              int32
home_club_manager_name         object
away_club_manager_name         object
Stadium_name                   object
attendance                      int32
referee                        object
home_club_formation            object
away_club_formation            object
home_club_name                 object
away_club_name                 object
aggregate                      object
competition_type               object
ï»¿game_lineups_id             object
lineup_type                    object
Players_shirt_number            int32
team_captain                    int32
game_event_id                  object
minute                          int32
game_event_type                object
description                    object
player_in_id

# dropping columns which have too many null values

In [49]:
df = df.drop(columns = ['ï»¿game_lineups_id', 'lineup_type', 'description', 'player_assist_id', 'player_code', 'Players_shirt_number', 'team_captain'])
print(df.dtypes.to_string())

game_id                         int32
season                          int32
round                          object
home_club_goals                 int32
away_club_goals                 int32
home_club_position              int32
away_club_position              int32
home_club_manager_name         object
away_club_manager_name         object
Stadium_name                   object
attendance                      int32
referee                        object
home_club_formation            object
away_club_formation            object
home_club_name                 object
away_club_name                 object
aggregate                      object
competition_type               object
game_event_id                  object
minute                          int32
game_event_type                object
player_in_id                   object
appearance_id                  object
yellow_cards                    int32
red_cards                       int32
goals                           int32
assists     

In [50]:
df = df.drop(columns = ['home_club_formation', 'away_club_formation'])

# Dropped these two columns as they contain empty rows

In [51]:
df.isnull().sum()

game_id                           0
season                            0
round                           933
home_club_goals                   0
away_club_goals                   0
home_club_position                0
away_club_position                0
home_club_manager_name          933
away_club_manager_name          933
Stadium_name                    933
attendance                        0
referee                         933
home_club_name                  933
away_club_name                  933
aggregate                       933
competition_type                933
game_event_id                  2658
minute                            0
game_event_type                2658
player_in_id                   2658
appearance_id                   933
yellow_cards                      0
red_cards                         0
goals                             0
assists                           0
minutes_played                    0
players_last_season               0
current_club_id             

In [52]:
df.shape

(4742, 42)

# Checking if there are any empty spaces which are not considered as null values in python

In [53]:
columns_to_check = ['home_club_manager_name', 'away_club_manager_name', 'referee', 'home_club_name', 'away_club_name', 'foot', 'agent_name', 'player_in_id']
empty_or_whitespace = df[columns_to_check].apply(lambda col: col.str.strip().eq(''))
empty_or_whitespace

,home_club_manager_name,away_club_manager_name,referee,home_club_name,away_club_name,foot,agent_name,player_in_id
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
4737,False,False,False,False,False,False,False,False
4738,False,False,False,False,False,False,False,False
4739,False,False,False,False,False,False,False,False
4740,False,False,False,False,False,False,True,False


# Replacing the empty spaces with nan/null values to replace them later

In [54]:
df[columns_to_check] = df[columns_to_check].replace(r'^\s*$', np.nan, regex=True)
df[columns_to_check].isnull().sum()

home_club_manager_name     939
away_club_manager_name     939
referee                    938
home_club_name            1061
away_club_name            1001
foot                       977
agent_name                1997
player_in_id              3450
dtype: int64

In [55]:
df.isnull().sum()

game_id                           0
season                            0
round                           933
home_club_goals                   0
away_club_goals                   0
home_club_position                0
away_club_position                0
home_club_manager_name          939
away_club_manager_name          939
Stadium_name                    933
attendance                        0
referee                         938
home_club_name                 1061
away_club_name                 1001
aggregate                       933
competition_type                933
game_event_id                  2658
minute                            0
game_event_type                2658
player_in_id                   3450
appearance_id                   933
yellow_cards                      0
red_cards                         0
goals                             0
assists                           0
minutes_played                    0
players_last_season               0
current_club_id             

# Replacing the null values with mode/'not available'

In [56]:
df['round'] = df['round'].fillna('Not Available')
df.isnull().sum()

game_id                           0
season                            0
round                             0
home_club_goals                   0
away_club_goals                   0
home_club_position                0
away_club_position                0
home_club_manager_name          939
away_club_manager_name          939
Stadium_name                    933
attendance                        0
referee                         938
home_club_name                 1061
away_club_name                 1001
aggregate                       933
competition_type                933
game_event_id                  2658
minute                            0
game_event_type                2658
player_in_id                   3450
appearance_id                   933
yellow_cards                      0
red_cards                         0
goals                             0
assists                           0
minutes_played                    0
players_last_season               0
current_club_id             

In [80]:
df['player_in_id'].fillna(-1, inplace=True)

In [57]:
columns_to_fill = 'home_club_manager_name', 'away_club_manager_name', 'Stadium_name', 'referee', 'home_club_name', 'away_club_name', 'agent_name', 'Player_name'

for col in columns_to_fill:
    df[col]=df[col].fillna('Not_Available')
df.head()

,game_id,season,round,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,Stadium_name,...,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,competition_id,date_id,players_id,Player_name,players_position
0,2222734,2012,19. Matchday,2,3,11,10,Miodrag Bozovic,Slavoljub Muslin,Olimp-2,...,175,1000000,500000,,Prosport,RU1,07-12-2012,104203,Eugene Starikov,Attack
1,2598832,2015,9. Matchday,0,0,11,5,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175,1000000,500000,,Prosport,UKR1,27-09-2015,104203,Eugene Starikov,Attack
2,2222798,2012,24. Matchday,1,1,4,11,Kurban Berdyev,Miodrag Bozovic,Central Stadium Kazan,...,175,1000000,500000,,Prosport,RU1,14-04-2013,104203,Eugene Starikov,Attack
3,2619433,2015,last 16 1st leg,0,1,9,9,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175,1000000,500000,,Prosport,UKRP,23-09-2015,104203,Eugene Starikov,Attack
4,2598841,2015,10. Matchday,2,0,2,11,Mircea Lucescu,Oleksandr Babych,Arena Lviv,...,175,1000000,500000,,Prosport,UKR1,03-10-2015,104203,Eugene Starikov,Attack


In [58]:
df.isnull().sum()

game_id                           0
season                            0
round                             0
home_club_goals                   0
away_club_goals                   0
home_club_position                0
away_club_position                0
home_club_manager_name            0
away_club_manager_name            0
Stadium_name                      0
attendance                        0
referee                           0
home_club_name                    0
away_club_name                    0
aggregate                       933
competition_type                933
game_event_id                  2658
minute                            0
game_event_type                2658
player_in_id                   3450
appearance_id                   933
yellow_cards                      0
red_cards                         0
goals                             0
assists                           0
minutes_played                    0
players_last_season               0
current_club_id             

In [61]:
columns_fill_mode = 'aggregate', 'competition_type', 'game_event_id', 'game_event_type', 'appearance_id', 'players_birth_country', 'sub_position', 'foot', 'competition_id', 'players_position'

for col in columns_fill_mode:
    mode_value = df[col].mode()[0]
    df[col].fillna(mode_value, inplace=True)

In [64]:
df.head(10)

,game_id,season,round,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,Stadium_name,...,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,competition_id,date_id,players_id,Player_name,players_position
0,2222734,2012,19. Matchday,2,3,11,10,Miodrag Bozovic,Slavoljub Muslin,Olimp-2,...,175,1000000,500000,,Prosport,RU1,07-12-2012,104203,Eugene Starikov,Attack
1,2598832,2015,9. Matchday,0,0,11,5,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175,1000000,500000,,Prosport,UKR1,27-09-2015,104203,Eugene Starikov,Attack
2,2222798,2012,24. Matchday,1,1,4,11,Kurban Berdyev,Miodrag Bozovic,Central Stadium Kazan,...,175,1000000,500000,,Prosport,RU1,14-04-2013,104203,Eugene Starikov,Attack
3,2619433,2015,last 16 1st leg,0,1,9,9,Oleksandr Babych,Vasyl Sachko,Stadion Chornomorets (2011),...,175,1000000,500000,,Prosport,UKRP,23-09-2015,104203,Eugene Starikov,Attack
4,2598841,2015,10. Matchday,2,0,2,11,Mircea Lucescu,Oleksandr Babych,Arena Lviv,...,175,1000000,500000,,Prosport,UKR1,03-10-2015,104203,Eugene Starikov,Attack
5,2335153,2013,6. Matchday,1,2,16,1,Anatoliy Davydov,Leonid Slutski,Trud,...,175,1000000,500000,,Prosport,RU1,24-08-2013,104203,Eugene Starikov,Attack
6,2222814,2012,25. Matchday,0,3,11,8,Miodrag Bozovic,Stanislav Cherchesov,Olimp-2,...,175,1000000,500000,,Prosport,RU1,22-04-2013,104203,Eugene Starikov,Attack
7,2598544,2015,8. Matchday,1,1,13,11,Vyacheslav Groznyi,Oleksandr Babych,Avangard,...,175,1000000,500000,,Prosport,UKR1,20-09-2015,104203,Eugene Starikov,Attack
8,2603931,2015,18. Matchday,4,0,3,12,Yuriy Vernydub,Oleksandr Babych,Slavutych Arena,...,175,1000000,500000,,Prosport,UKR1,12-03-2016,104203,Eugene Starikov,Attack
9,2222707,2012,16. Matchday,0,0,2,12,Guus Hiddink,Miodrag Bozovic,Dinamo,...,175,1000000,500000,,Prosport,RU1,18-11-2012,104203,Eugene Starikov,Attack


In [65]:
df.isnull().sum()

game_id                           0
season                            0
round                             0
home_club_goals                   0
away_club_goals                   0
home_club_position                0
away_club_position                0
home_club_manager_name            0
away_club_manager_name            0
Stadium_name                      0
attendance                        0
referee                           0
home_club_name                    0
away_club_name                    0
aggregate                         0
competition_type                  0
game_event_id                     0
minute                            0
game_event_type                   0
player_in_id                   3450
appearance_id                     0
yellow_cards                      0
red_cards                         0
goals                             0
assists                           0
minutes_played                    0
players_last_season               0
current_club_id             

In [67]:
df.dtypes

game_id                         int32
season                          int32
round                          object
home_club_goals                 int32
away_club_goals                 int32
home_club_position              int32
away_club_position              int32
home_club_manager_name         object
away_club_manager_name         object
Stadium_name                   object
attendance                      int32
referee                        object
home_club_name                 object
away_club_name                 object
aggregate                      object
competition_type               object
game_event_id                  object
minute                          int32
game_event_type                object
player_in_id                   object
appearance_id                  object
yellow_cards                    int32
red_cards                       int32
goals                           int32
assists                         int32
minutes_played                  int32
players_last

# Changing the datatype from object to datetime while filling the null values

In [68]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')
df['date_of_birth'].fillna(pd.Timestamp('11-12-1987'), inplace=True)

In [69]:
df['contract_expiration_date'].mode()

0    
Name: contract_expiration_date, dtype: object

In [70]:
df['contract_expiration_date'] = pd.to_datetime(df['contract_expiration_date'], errors='coerce')
df['contract_expiration_date'].fillna(pd.Timestamp('30-06-2027'), inplace=True)

In [71]:
df['date_id'].mode()

0    03-11-2012
Name: date_id, dtype: object

In [72]:
df['date_id'] = pd.to_datetime(df['date_id'], errors = 'coerce')
df['date_id'].fillna(pd.Timestamp('03-11-2012'), inplace=True)

In [73]:
df.isnull().sum()

game_id                           0
season                            0
round                             0
home_club_goals                   0
away_club_goals                   0
home_club_position                0
away_club_position                0
home_club_manager_name            0
away_club_manager_name            0
Stadium_name                      0
attendance                        0
referee                           0
home_club_name                    0
away_club_name                    0
aggregate                         0
competition_type                  0
game_event_id                     0
minute                            0
game_event_type                   0
player_in_id                   3450
appearance_id                     0
yellow_cards                      0
red_cards                         0
goals                             0
assists                           0
minutes_played                    0
players_last_season               0
current_club_id             

In [74]:
df.dtypes

game_id                                 int32
season                                  int32
round                                  object
home_club_goals                         int32
away_club_goals                         int32
home_club_position                      int32
away_club_position                      int32
home_club_manager_name                 object
away_club_manager_name                 object
Stadium_name                           object
attendance                              int32
referee                                object
home_club_name                         object
away_club_name                         object
aggregate                              object
competition_type                       object
game_event_id                          object
minute                                  int32
game_event_type                        object
player_in_id                           object
appearance_id                          object
yellow_cards                      

# Converting object datatype columns to string

In [75]:
convert_string = ['round', 'home_club_manager_name', 'away_club_manager_name', 'Stadium_name', 'referee', 'home_club_name', 'away_club_name', 'competition_type']

df[convert_string] = df[convert_string].astype(pd.StringDtype())

In [76]:
convert_string2 = ['game_event_id', 'game_event_type', 'appearance_id', 'players_birth_country', 'sub_position', 'foot', 'agent_name', 'competition_id', 'Player_name', 'players_position']

df[convert_string2] = df[convert_string2].astype(pd.StringDtype())

In [84]:
df['player_in_id'] = df['player_in_id'].astype(int)

In [85]:
df.dtypes

game_id                                 int32
season                                  int32
round                          string[python]
home_club_goals                         int32
away_club_goals                         int32
home_club_position                      int32
away_club_position                      int32
home_club_manager_name         string[python]
away_club_manager_name         string[python]
Stadium_name                   string[python]
attendance                              int32
referee                        string[python]
home_club_name                 string[python]
away_club_name                 string[python]
aggregate                              object
competition_type               string[python]
game_event_id                  string[python]
minute                                  int32
game_event_type                string[python]
player_in_id                            int32
appearance_id                  string[python]
yellow_cards                      

In [86]:
df.isnull().sum()

game_id                        0
season                         0
round                          0
home_club_goals                0
away_club_goals                0
home_club_position             0
away_club_position             0
home_club_manager_name         0
away_club_manager_name         0
Stadium_name                   0
attendance                     0
referee                        0
home_club_name                 0
away_club_name                 0
aggregate                      0
competition_type               0
game_event_id                  0
minute                         0
game_event_type                0
player_in_id                   0
appearance_id                  0
yellow_cards                   0
red_cards                      0
goals                          0
assists                        0
minutes_played                 0
players_last_season            0
current_club_id                0
players_birth_country          0
date_of_birth                  0
sub_positi

In [88]:
df.to_excel('CapstoneProject_Football_Merged_Clean_data.xlsx', index = False)